In [4]:
import os
import numpy as np
import pandas as pd
from mpc_utils import load_train_test_data_27s, load_train_test_data

In [5]:
x_train, y_train, x_test, y_test = load_train_test_data_27s(include_L=True)
l_train, l_test = x_train[:, :, 7:], x_test[:, :, 7:]
x_train, x_test = x_train[:, :, :7], x_test[:, :, :7]
x_train.shape, y_train.shape, x_test.shape, y_test.shape, l_train.shape

((200, 1350, 7), (200, 1350, 1), (25, 1350, 7), (25, 1350, 1), (200, 1350, 1))

In [6]:
from models.lmu_torch import LMUModel
import torch

def train_model(x_train, y_train, x_test, y_test, input_size, hidden_size, memory_size, theta, epochs=100, batch_size=16, lr=1e-3, device='cpu'):
    model = LMUModel(input_size=input_size, hidden_size=hidden_size, memory_size=memory_size, output_size=1, theta=theta, device=device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss = torch.nn.MSELoss()
    n_batches = x_train.shape[0] // batch_size

    for epoch in range(epochs+1):
        # eval
        if epoch % 50 == 0:
            loss_train = []
            for batch_idx in range(n_batches-1):
                model.eval()
                x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
                y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
                x = torch.tensor(x, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32, device=device)
                ypr = model(x)
                loss_train.append(loss(ypr, y).item())
            loss_test = []
            for batch_idx in range(max(1, x_test.shape[0] // batch_size)):
                model.eval()
                x = x_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
                y = y_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
                x = torch.tensor(x, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32, device=device)
                ypr = model(x)
                loss_test.append(loss(ypr, y).item())
            print(epoch, 'train', np.array(loss_train).mean(), 'test', np.array(loss_test).mean())

        # train
        epoch_loss = []
        model.train()
        for batch_idx in range(n_batches-1):
            x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            x = torch.tensor(x, dtype=torch.float32, device=device)
            y = torch.tensor(y, dtype=torch.float32, device=device)

            optimizer.zero_grad()
            y_pred = model(x)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()

            epoch_loss.append(l.item())

        # log training loss
        avg_epoch_loss = np.array(epoch_loss).mean()
        if epoch % 10 == 0:
            print(epoch, avg_epoch_loss)
        else:
            print(epoch, avg_epoch_loss, end='\r')
    return model

In [4]:
hidden_size = 64
memory_size = 32
theta = 16

length_model = train_model(x_train, l_train, x_test, l_test, 7, hidden_size, memory_size, theta, epochs=100, batch_size=16, lr=1e-3)
model_name = f'mpc_models/lmu_length_7-1-{hidden_size}-{memory_size}-{theta}.pt'
torch.save(length_model.state_dict(), model_name)

0 train 0.1053507687015967 test 0.09588231891393661
0 0.09099637581543489
10 0.02335170233114199
20 0.012840221043337475
30 0.009863007136366585
40 0.009216473099182953
50 train 0.009070090695538303 test 0.01331960316747427
50 0.009083957496014509
60 0.009032332169061357
70 0.008989603766663508
80 0.008944488257508387
90 0.008893085643649101
100 train 0.008791436483575539 test 0.01313480269163847
100 0.008832149588587608


In [9]:
xtrl = np.concatenate([x_train, l_train], axis=2)
xtel = np.concatenate([x_test, l_test], axis=2)
action_model_true_length = train_model(xtrl, y_train, xtel, y_test, 8, hidden_size, memory_size, theta, epochs=250, batch_size=16, lr=1e-3)
model_name = f'mpc_models/lmu_action_true_7-1-{hidden_size}-{memory_size}-{theta}_250epochs.pt'
torch.save(action_model_true_length.state_dict(), model_name)

0 train 0.49338705973191693 test 0.4251982569694519
0 0.3003939932042902
10 0.030197971246459267
20 0.017432937398552895
30 0.013452758758582851
40 0.011286370117555965
50 train 0.009826862337914381 test 0.011015781201422215
50 0.009889701804654165
60 0.008871115955778143
70 0.008098993958397345
80 0.0075436259450560265
90 0.007089623715728521
100 train 0.006654849614609371 test 0.007684497162699699
100 0.006732017733156681
110 0.0064287441359324885
120 0.006165519145063378
130 0.0059427381916479635
140 0.0057501419108699665
150 train 0.005503616719083352 test 0.006301489192992449
150 0.005579321433536031
160 0.0054474276998503644
170 0.0052666694179854615
180 0.0050775121304799214
190 0.0049805663187395445
200 train 0.0049021320459856224 test 0.005426288582384586
200 0.004923061510040002
210 0.0047615041786974125
220 0.0046357327479530475
230 0.004675860918888991
240 0.004516785710372708
250 train 0.004872065418484536 test 0.005519535858184099
250 0.004748562583699822


In [14]:
hidden_size = 64
memory_size = 32
theta = 16

lmodel = LMUModel(input_size=7, output_size=1, hidden_size=hidden_size,
                  memory_size=memory_size, theta=theta)
lmodel.load_state_dict(torch.load('mpc_models/lmu_length_7-1-64-32-16.pt'))

amodel = LMUModel(input_size=8, output_size=1, hidden_size=hidden_size,
                  memory_size=memory_size, theta=theta)
amodel.load_state_dict(torch.load('mpc_models/lmu_action_true_7-1-64-32-16_250epochs.pt'))

<All keys matched successfully>